In [92]:
import pandas as pd
import numpy as np
from string import digits
import math

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [75]:
train=pd.read_csv('Train.csv')
test_set=pd.read_csv('Test.csv')
combine=pd.concat([train,test_set],ignore_index=True)

combine

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.300,1999,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.920,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.500,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.200,1998,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.930,1987,OUT013,Tier 3,High,Supermarket Type1
5,Regular,FDP36,51.4008,556.6088,Baking Goods,0.000000,10.395,2009,OUT018,Tier 3,Medium,Supermarket Type2
6,Regular,FDO10,57.6588,343.5528,Snack Foods,0.012741,13.650,1987,OUT013,Tier 3,High,Supermarket Type1
7,Low Fat,FDP10,107.7622,4022.7636,Snack Foods,0.127470,NaN,1985,OUT027,Tier 3,Medium,Supermarket Type3
8,Regular,FDH17,96.9726,1076.5986,Frozen Foods,0.016687,16.200,2002,OUT045,Tier 2,NaN,Supermarket Type1
9,Regular,FDU28,187.8214,4710.5350,Frozen Foods,0.094450,19.200,2007,OUT017,Tier 2,NaN,Supermarket Type1


In [76]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 12 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  11765 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  10188 non-null object
Outlet_Type                  14204 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB


In [77]:
combine.Item_Identifier.value_counts()[:10]

FDR09    10
FDT04    10
FDW56    10
DRH11    10
FDW24    10
DRD15    10
DRF15    10
FDJ41    10
FDY03    10
FDV20    10
Name: Item_Identifier, dtype: int64

In [78]:
combine.Outlet_Identifier.value_counts()

OUT027    1559
OUT013    1553
OUT049    1550
OUT035    1550
OUT046    1550
OUT045    1548
OUT018    1546
OUT017    1543
OUT010     925
OUT019     880
Name: Outlet_Identifier, dtype: int64

In [79]:
#let's fill missing outlet size data. Maybe some entries 
combine[combine.Outlet_Size.isnull()].Outlet_Identifier.value_counts()

#These three outlets failed to include any info on store size

OUT045    1548
OUT017    1543
OUT010     925
Name: Outlet_Identifier, dtype: int64

In [80]:
OutletInfo=combine[['Outlet_Establishment_Year','Outlet_Location_Type','Outlet_Type']]
print(OutletInfo[combine.Outlet_Identifier=='OUT045'].apply(pd.Series.value_counts))
print(OutletInfo[combine.Outlet_Identifier=='OUT017'].apply(pd.Series.value_counts))
print(OutletInfo[combine.Outlet_Identifier=='OUT010'].apply(pd.Series.value_counts))

# looks like the info on each outlet is consistent

                   Outlet_Establishment_Year  Outlet_Location_Type  \
2002                                  1548.0                   NaN   
Tier 2                                   NaN                1548.0   
Supermarket Type1                        NaN                   NaN   

                   Outlet_Type  
2002                       NaN  
Tier 2                     NaN  
Supermarket Type1       1548.0  
                   Outlet_Establishment_Year  Outlet_Location_Type  \
2007                                  1543.0                   NaN   
Tier 2                                   NaN                1543.0   
Supermarket Type1                        NaN                   NaN   

                   Outlet_Type  
2007                       NaN  
Tier 2                     NaN  
Supermarket Type1       1543.0  
               Outlet_Establishment_Year  Outlet_Location_Type  Outlet_Type
1998                               925.0                   NaN          NaN
Tier 3                

/root/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:5539: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)


In [81]:
size={'Small':1.0,'Medium':2.0,'High':3.0}

In [82]:
#Let's use KNN to solve for missing store sizes
TData=pd.DataFrame()
TData=pd.concat([pd.get_dummies(combine['Outlet_Type']),
      pd.get_dummies(combine['Outlet_Location_Type'])], axis=1)
TData['Est_Year']=combine['Outlet_Establishment_Year']
TData['Outlet_Size']=combine['Outlet_Size'].map(size)
TData_out_X_test=TData[TData.Outlet_Size.isnull()]
TData_out_X_train=TData[TData.Outlet_Size.notnull()]

X_tr,X_te,y_tr, y_te=train_test_split(TData_out_X_train.drop('Outlet_Size',axis=1),TData_out_X_train['Outlet_Size'])

knn=KNeighborsClassifier()
knn.fit(X_tr,y_tr)
print('train score: {}'.format(knn.score(X_tr,y_tr)))
print('test score: {}'.format(knn.score(X_te,y_te)))

TData['Size_predict']=knn.predict(TData.drop('Outlet_Size',axis=1))
TData['Outlet_Size']=TData['Outlet_Size'].fillna(value=TData.Size_predict)

train score: 1.0
test score: 1.0


In [83]:
TData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 10 columns):
Grocery Store        14204 non-null uint8
Supermarket Type1    14204 non-null uint8
Supermarket Type2    14204 non-null uint8
Supermarket Type3    14204 non-null uint8
Tier 1               14204 non-null uint8
Tier 2               14204 non-null uint8
Tier 3               14204 non-null uint8
Est_Year             14204 non-null int64
Outlet_Size          14204 non-null float64
Size_predict         14204 non-null float64
dtypes: float64(2), int64(1), uint8(7)
memory usage: 430.1 KB


In [84]:
IData=pd.DataFrame()
IData=pd.concat([pd.get_dummies(combine['Item_Fat_Content']),
                 pd.get_dummies(combine['Item_Type'])], axis=1)
IData['MRP']=combine.Item_MRP
IData['Visibility']=combine.Item_Visibility
IData['Weight']=combine.Item_Weight.fillna(combine.Item_Weight.mean())
IData['Sales']=combine.Item_Outlet_Sales

Data=pd.concat([IData,TData],axis=1)
Data['Est_Year']=2013-TData.Est_Year
Data.info()

train=Data[Data['Sales'].notnull()]
test=Data[Data['Sales'].isnull()].drop('Sales',axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 35 columns):
LF                       14204 non-null uint8
Low Fat                  14204 non-null uint8
Regular                  14204 non-null uint8
low fat                  14204 non-null uint8
reg                      14204 non-null uint8
Baking Goods             14204 non-null uint8
Breads                   14204 non-null uint8
Breakfast                14204 non-null uint8
Canned                   14204 non-null uint8
Dairy                    14204 non-null uint8
Frozen Foods             14204 non-null uint8
Fruits and Vegetables    14204 non-null uint8
Hard Drinks              14204 non-null uint8
Health and Hygiene       14204 non-null uint8
Household                14204 non-null uint8
Meat                     14204 non-null uint8
Others                   14204 non-null uint8
Seafood                  14204 non-null uint8
Snack Foods              14204 non-null uint8
Soft Drinks      

In [85]:
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 35 columns):
LF                       8523 non-null uint8
Low Fat                  8523 non-null uint8
Regular                  8523 non-null uint8
low fat                  8523 non-null uint8
reg                      8523 non-null uint8
Baking Goods             8523 non-null uint8
Breads                   8523 non-null uint8
Breakfast                8523 non-null uint8
Canned                   8523 non-null uint8
Dairy                    8523 non-null uint8
Frozen Foods             8523 non-null uint8
Fruits and Vegetables    8523 non-null uint8
Hard Drinks              8523 non-null uint8
Health and Hygiene       8523 non-null uint8
Household                8523 non-null uint8
Meat                     8523 non-null uint8
Others                   8523 non-null uint8
Seafood                  8523 non-null uint8
Snack Foods              8523 non-null uint8
Soft Drinks              8523 non-null

(None, None)

In [86]:
X_train,X_test,y_train,y_test=train_test_split(train.drop(['Sales'],axis=1),train.Sales)

In [95]:
lr=GradientBoostingRegressor(max_features=15,min_samples_split=30)
lr.fit(X_train,y_train)
train_pred=lr.predict(X_train)
test_pred=lr.predict(X_test)
print('train score: {}'.format(lr.score(X_train,y_train)))
print('test score: {}'.format(lr.score(X_test,y_test)))
rms_train=math.sqrt(mean_squared_error(y_train,train_pred))
rms_test=math.sqrt(mean_squared_error(y_test,test_pred))
print('train set RMS score: {}'.format(rms_train))
print('test set RMS score: {}'.format(rms_test))

train score: 0.6254117694788965
test score: 0.6197480234970545
train set RMS score: 1043.1254741941705
test set RMS score: 1056.0194629698465


In [88]:
param_grid={'max_features':[1,5,10,15,20,25,30],'min_samples_split':[10,20,30,40]}
grid_search=GridSearchCV(GradientBoostingRegressor(),param_grid,cv=5)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'max_features': 15, 'min_samples_split': 30}
0.587150554099


In [ ]:
param_grid={'max_features':[1,5,10,15,20,25,30],'min_samples_split':[10,20,30,40]}
grid_search=GridSearchCV(GradientBoostingRegressor(),param_grid,cv=5)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [91]:
#fin=GradientBoostingRegressor(max_features=15,min_samples_split=30)
fin.fit(train.drop(['Sales'],axis=1),train.Sales)
pred=fin.predict(test)

solution=pd.DataFrame(data={'Item_Identifier':list(test_set.Item_Identifier),'Outlet_Identifier':list(test_set.Outlet_Identifier),'Item_Outlet_Sales':pred})
solution.to_csv('solutions.csv',index=False)